In [1]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from imutils import paths
import random
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import flwr as fl
# Load and compile Keras model


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
BATCH_SIZE = 8
train_data_dir = 'D:/TQ/Balanced_Another/Client5/train/'

test_data_dir = 'D:/TQ/Balanced_Another/Client5/val/'


In [11]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split percentage
)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)




In [12]:
print("Class Counts in Training Samples:")
total_classes = sorted(os.listdir(train_data_dir))
print(total_classes)
for category in total_classes:
    category_path = os.path.join(train_data_dir, category)
    sample_count = len(os.listdir(category_path))
    print(f"{category}: {sample_count} samples")


Class Counts in Training Samples:
['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
Animal_Abuse: 124 samples
Arson: 138 samples
Fight: 342 samples
Normal: 199 samples
Riot: 253 samples
Traffic_Accident: 179 samples
Train_Accident: 0 samples


In [13]:
train_classes = sorted(os.listdir(train_data_dir))
test_classes = sorted(os.listdir(test_data_dir))


In [25]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the base model
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer with a dropout rate of 0.5
outputs = Dense(7, activation='softmax')(x)  # Assuming binary classification

# Create the full model
model1 = Model(inputs, outputs)
optimizer = Adam(learning_rate=0.001)
model1.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [14]:
traingen = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(225, 225),
                                                   class_mode='categorical',
                                                   classes=total_classes,
                                                   subset='training',
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True,
                                                   seed=42)

validgen = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(225, 225),
                                               class_mode='categorical',
                                               classes=total_classes,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                             target_size=(225, 225),
                                             class_mode='categorical',
                                             classes=total_classes,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)


Found 992 images belonging to 7 classes.
Found 243 images belonging to 7 classes.
Found 349 images belonging to 7 classes.


In [27]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
callbacks = [early_stopping]


In [28]:
history = model1.fit(
            traingen,
            epochs=5,
            steps_per_epoch=len(traingen),
            validation_data=validgen,
            validation_steps=len(validgen),
            callbacks=callbacks,
            verbose = 1
)

Epoch 1/5


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


124/124 [==============================] - 26s 194ms/step - loss: 0.9316 - accuracy: 0.6925 - val_loss: 1.8100 - val_accuracy: 0.4239
Epoch 2/5
124/124 [==============================] - 22s 179ms/step - loss: 0.2741 - accuracy: 0.9062 - val_loss: 1.9967 - val_accuracy: 0.4897
Epoch 3/5
124/124 [==============================] - 22s 180ms/step - loss: 0.1775 - accuracy: 0.9446 - val_loss: 2.4414 - val_accuracy: 0.5226
Epoch 4/5
124/124 [==============================] - 21s 173ms/step - loss: 0.1483 - accuracy: 0.9516 - val_loss: 2.1008 - val_accuracy: 0.4979
Epoch 5/5
124/124 [==============================] - 22s 178ms/step - loss: 0.1047 - accuracy: 0.9758 - val_loss: 2.4676 - val_accuracy: 0.4527


In [29]:
#model = tf.keras.models.load_model("D:/TQ/Federated/Client5_Centralized.h5")

In [30]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = model1.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

349/349 [==============================] - 7s 19ms/step


In [31]:
print("Accuracy:")
test_loss, test_accuracy = model.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Accuracy:
Test Loss: 2.3387770652770996
Test Accuracy: 0.5644699335098267


In [32]:
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Confusion Matrix Before Aggregation:
[[ 1  0  1 28  1  0  0]
 [ 0 28  0  7  0  0  0]
 [ 0  0 49 36  1  0  0]
 [ 0  0  0 45  0  5  0]
 [ 0  0  1 33 30  0  0]
 [ 0  0  0  4  0 41  0]
 [ 0  0  2 34  0  2  0]]


In [33]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm2, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix5.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [24]:
model.save("Client5_Centralized.h5")

In [17]:
client5_center = tf.keras.models.load_model('Client5_Centralized.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [18]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client5_center.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


349/349 [==============================] - 6s 17ms/step


In [20]:
print("Accuracy:")
test_loss, test_accuracy = client5_center.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Accuracy:
Test Loss: 2.4030911922454834
Test Accuracy: 0.5702005624771118
Confusion Matrix Before Aggregation:
[[ 2  0  1 28  0  0  0]
 [ 0 30  1  4  0  0  0]
 [ 0  4 49 31  2  0  0]
 [ 0  0  0 45  0  5  0]
 [ 0  0  0 33 31  0  0]
 [ 0  0  0  2  0 43  0]
 [ 0  0  0 34  0  4  0]]


In [25]:
from sklearn.metrics import classification_report 
print(classification_report(y_true, y_pred_labels, target_names=total_classes))


                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.95      0.98        42
           Arson       0.86      0.96      0.91        26
           Fight       1.00      1.00      1.00        22
          Normal       0.69      0.86      0.77        50
            Riot       1.00      0.93      0.97       105
Traffic_Accident       0.85      1.00      0.92        35
  Train_Accident       0.00      0.00      0.00        12

        accuracy                           0.90       292
       macro avg       0.77      0.82      0.79       292
    weighted avg       0.88      0.90      0.89       292



C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [21]:
client5 = tf.keras.models.load_model('client5_30_rounds.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [22]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client5.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels
test_loss, test_accuracy = client5.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

print(classification_report(y_true, y_pred_labels, target_names=total_classes))


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


349/349 [==============================] - 6s 17ms/step
Test Loss: 1.5360695123672485
Test Accuracy: 0.7134670615196228
                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.74      0.85        31
           Arson       1.00      0.69      0.81        35
           Fight       0.97      0.44      0.61        86
          Normal       0.32      0.92      0.48        50
            Riot       0.88      0.56      0.69        64
Traffic_Accident       0.91      0.93      0.92        45
  Train_Accident       1.00      0.87      0.93        38

        accuracy                           0.69       349
       macro avg       0.87      0.74      0.76       349
    weighted avg       0.86      0.69      0.72       349



In [23]:
cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm)

Confusion Matrix Before Aggregation:
[[23  0  0  8  0  0  0]
 [ 0 24  0 11  0  0  0]
 [ 0  0 38 43  5  0  0]
 [ 0  0  0 46  0  4  0]
 [ 0  0  1 27 36  0  0]
 [ 0  0  0  3  0 42  0]
 [ 0  0  0  5  0  0 33]]


In [32]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix5f.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx
